# Setup

In [1]:
# Installs

%pip install -qU langchain-cohere langchain-community langchain-core==0.2.40 langchain_experimental langchain-google-community langchain-google-genai langchain-huggingface==0.0.3 langchain-openai langchain-qdrant langchain-google-vertexai
%pip install -qU -q chainlit==1.1.302
%pip install -qU -q docx2txt
%pip install -qU google-cloud-aiplatform
%pip install -qU google-cloud-discoveryengine
%pip install -qU IProgress
%pip install -qU nltk
%pip install -qU openpyxl
%pip install -qU pymupdf
%pip install -qU python-dotenv
%pip install -qU ragas==0.1.20
%pip install -qU tqdm
%pip install -qU unstructured 

# Verify installed packages have compatible dependencies
%pip check

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
langchain-ollama 0.2.0 has requirement langchain-core<0.4.0,>=0.3.0, but you have langchain-core 0.2.40.
langserve 0.3.0 has requirement langc

In [2]:
# Download punkt_tab module that is used for sentence tokenizaiton

import nltk

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /Users/david/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

# Environment Variables

In [3]:
# Get environment variables

import os
from dotenv import load_dotenv
import uuid

# Load environment variables from .env file
load_dotenv()

True

# Google Auth

In [4]:
# Set Google user permissions

import os
import sys

# If running in Colab, use the permissions of the currently authenticated user
if 'google.colab' in sys.modules:
    print('Running in Google Colab')
    from google.colab import auth

    auth.authenticate_user()

# If not, set the GOOGLE_APPLICATION_CREDENTIALS to the service account credentials file 
else:
    print('Running locally')
    # os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'credentials.json'
    # print(os.environ['GOOGLE_APPLICATION_CREDENTIALS'])

#####

def test_google_perms():
    from google.cloud import storage

    # Now, you can use the Google Cloud client libraries
    client = storage.Client()

    # List all buckets in your project
    buckets = list(client.list_buckets())
    print(buckets)

test_google_perms()

Running locally
[<Bucket: 395640738565_us_import_content_with_faq_csv>, <Bucket: aie4-demo-docs>, <Bucket: aie4-demo-medi-cal-docs>]


In [5]:
# Initialize Vertex AI

import vertexai

vertexai.init(project=os.environ['PROJECT_ID'], location=os.environ['REGION'])

In [6]:
# Verfify that our Google API key works

import requests

GOOGLE_API_KEY = os.environ['GOOGLE_API_KEY']
url = f'https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash-latest:generateContent?key={GOOGLE_API_KEY}'
  
headers = {
    "Content-Type": "application/json",
}
  
data = {"contents":[{"parts":[{"text":"Explain how AI works"}]}]}

response = requests.post(url, headers=headers, json=data)
print(response.text)

{
  "candidates": [
    {
      "content": {
        "parts": [
          {
            "text": "Artificial intelligence (AI) is a broad field encompassing various techniques that enable computers to mimic human intelligence.  There's no single \"how it works\" answer, as different AI approaches use different methods. However, we can break it down into key concepts:\n\n**1. Data is King:**  AI systems learn from data. The more relevant and high-quality data they are trained on, the better they perform. This data can be anything from images and text to sensor readings and financial transactions.\n\n**2. Algorithms and Models:**  AI uses algorithms – sets of rules and statistical techniques – to analyze data and identify patterns. These algorithms are often implemented as \"models,\" which are mathematical representations of the relationships within the data.  Examples include:\n\n* **Machine Learning (ML):** This is a subset of AI where systems learn from data without explicit programmi

# Utils

In [7]:
# Load the docs from a directory

# https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/file_directory/

from langchain_community.document_loaders import DirectoryLoader 
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import UnstructuredExcelLoader

def process_directory(path: str, glob: str, loader_cls: str, use_multithreading=True):
    
    loader = DirectoryLoader(path=path, glob=glob, show_progress=True, loader_cls=loader_cls, use_multithreading=use_multithreading)
    
    docs = loader.load()
    
    return docs

#####

def test_process_directory():
    docs = []
    
    docs_pdf = process_directory('docs/medi-cal/', '**/*.pdf', PyMuPDFLoader, True)
    # print(len(docs_pdf))
    docs_excel = process_directory('docs/medi-cal/', '**/*.xlsx', UnstructuredExcelLoader, True)
    # print(len(docs_excel))

    docs.extend(docs_pdf)
    docs.extend(docs_excel)
    print(len(docs))

test_process_directory()

  0%|          | 0/1 [00:00<?, ?it/s]/opt/anaconda3/envs/aie4/lib/python3.11/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/opt/anaconda3/envs/aie4/lib/python3.11/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
100%|██████████| 1/1 [00:01<00:00,  1.27s/it]

2477


In [8]:
# Load the docs from a single file

# TODO - add TextLoader refernece
# TODO - add CSVLoader reference
# https://python.langchain.com/docs/integrations/document_loaders/pymupdf/
# https://python.langchain.com/docs/integrations/document_loaders/microsoft_word/

from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import Docx2txtLoader

def process_file(path: str):

    docs = None

    # Select the right loader
    if 'txt' in path.lower():
        loader = TextLoader(path)
    elif 'pdf' in path.lower():
        loader = PyMuPDFLoader(path)
    elif 'docx' in path.lower():
        loader = Docx2txtLoader(path)
    else:
        print(f'No document loader found for {path}')

    docs = loader.load()

    return docs

#####

def test_process_file():
    doc = process_file('docs/medi-cal/acu.pdf')
    print(doc[0].metadata)

test_process_file()

{'source': 'docs/medi-cal/acu.pdf', 'file_path': 'docs/medi-cal/acu.pdf', 'page': 0, 'total_pages': 4, 'format': 'PDF 1.7', 'title': 'Acupuncture Services (acu)', 'author': 'Department of Healthcare Services (DHCS)', 'subject': '', 'keywords': 'acu_a01o11', 'creator': 'Microsoft® Word for Office 365', 'producer': 'Microsoft® Word for Office 365', 'creationDate': "D:20200731144705-07'00'", 'modDate': "D:20200731155009-07'00'", 'trapped': ''}


In [9]:
# Create embeddings using Hugging Face 

from langchain_huggingface import HuggingFaceEmbeddings

def create_embeddings_huggingface(model='Snowflake/snowflake-arctic-embed-m') -> HuggingFaceEmbeddings:

    # Initialize the OpenAIEmbeddings class
    embeddings = HuggingFaceEmbeddings(model_name=model)

    return embeddings

#####

def test_create_embeddings_huggingface():
    text = 'What is my benefit for acupuncture?'
    embeddings = create_embeddings_huggingface()
    vector = embeddings.embed_query(text)
    print(vector)
    return embeddings

test_create_embeddings_huggingface()

/opt/anaconda3/envs/aie4/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


[-0.018624262884259224, 0.07942049950361252, -0.01396368071436882, -0.019285650923848152, 0.019548555836081505, 0.04386259615421295, 0.001751587842591107, 0.003699950175359845, -0.035668954253196716, -0.03971784934401512, -0.012464510276913643, 0.020438581705093384, 0.017930110916495323, -0.0047818077728152275, -0.033828254789114, -0.010549304075539112, 0.06566156446933746, -0.01852135919034481, -0.024217918515205383, -0.028709201142191887, -0.02063765749335289, -0.01962468773126602, -0.004117606673389673, 0.03691602125763893, -0.023654839023947716, 0.03305622190237045, 0.03593701496720314, 0.05134712532162666, -0.07007365673780441, -0.0522121861577034, 0.029391692951321602, 0.03666972368955612, -0.017945537343621254, -0.06466532498598099, -0.018098600208759308, 0.007805246859788895, -0.0360274501144886, -0.03589756786823273, -0.0524902306497097, -0.00038497804780490696, -0.002180682262405753, 0.004602802451699972, 0.012346711941063404, 0.03780534490942955, -0.028559066355228424, -0.01

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='Snowflake/snowflake-arctic-embed-m', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [10]:
# Create embeddings using OpenAI

from langchain_openai import OpenAIEmbeddings

def create_embeddings_openai(model='text-embedding-ada-002') -> OpenAIEmbeddings:

    # Initialize the OpenAIEmbeddings class
    embeddings = OpenAIEmbeddings(model=model)

    return embeddings

#####

def test_create_embeddings_openai():
    text = 'What is my benefit for acupuncture?'
    embeddings = create_embeddings_openai()
    vector = embeddings.embed_query(text)
    print(vector)
    return embeddings

test_create_embeddings_openai()

[-0.013142690062522888, -0.0037146771792322397, 0.043068721890449524, -0.04360708221793175, -0.03262455761432648, 0.013970416970551014, -0.02534325048327446, -0.019892366603016853, -0.016298819333314896, 0.003105658572167158, -0.0008849276346154511, 0.010551837272942066, 0.003590181702747941, 0.004108352120965719, -0.005821006838232279, -0.0023082143161445856, 0.05133253335952759, -0.009205939248204231, 0.005363401956856251, 0.005289377644658089, -0.013849286362528801, 0.0024579453747719526, 0.0037819722201675177, -0.0010405470384284854, -0.014024253003299236, 0.018075404688715935, 0.010585484094917774, -0.012173643335700035, -0.021776622161269188, -0.02169586904346943, 0.03650074079632759, 0.016419950872659683, -0.009576060809195042, -0.008479154668748379, -0.012173643335700035, 0.02253032475709915, 0.00017570272029843181, -0.009892347268760204, 0.022678373381495476, -0.0004090266884304583, 0.00913864467293024, 0.0032452954910695553, -0.0010472765425220132, 0.00490243174135685, -0.006

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x35be7df10>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x35bc8df10>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [11]:
# Create embeddings using Vertex AI

# https://python.langchain.com/docs/integrations/text_embedding/google_vertex_ai_palm/

from langchain_google_vertexai import VertexAIEmbeddings

def create_embeddings_vertexai(model="text-embedding-004") -> VertexAIEmbeddings:

    # Initialize the VertexAIEmbeddings class
    embeddings = VertexAIEmbeddings(model_name=model)

    return embeddings

#####

def test_create_embeddings_vertexai():
    text = 'What is my benefit for acupuncture?'
    embeddings = create_embeddings_vertexai()
    vector = embeddings.embed_query(text)
    print(vector)
    return embeddings

test_create_embeddings_vertexai()

[0.037580445408821106, 0.033997565507888794, 0.006779782474040985, 0.013889092952013016, -0.01751645840704441, 0.05550519376993179, -0.007817636243999004, 0.012510783970355988, -0.03990676626563072, -0.0073811886832118034, -0.021110394969582558, 0.018350055441260338, 0.046891406178474426, 0.037558868527412415, -0.0948234349489212, -0.05031082406640053, -0.062025584280490875, -0.013999848626554012, -0.14326012134552002, -0.03631572052836418, 0.04239406809210777, 0.0033956223633140326, -0.012713606469333172, -0.023773323744535446, 0.0029369941912591457, 0.002878165803849697, -0.010965565219521523, 0.004967726767063141, 0.049845315515995026, -0.03375077620148659, -0.02053850144147873, 0.017944777384400368, 0.05141162872314453, -0.0443461574614048, 0.06225729361176491, 0.002635953249409795, 0.032474566251039505, 0.012381038628518581, 0.014667452313005924, -0.07240024954080582, -0.019168945029377937, -0.035043321549892426, -0.03396611660718918, 0.016433045268058777, -0.01277957670390606, -0

VertexAIEmbeddings(client=<vertexai.language_models.TextEmbeddingModel object at 0x3480e2910>, async_client=None, project='aie4-demo', location='us-central1', request_parallelism=5, max_retries=6, stop=None, model_name='text-embedding-004', full_model_name=None, client_options=ClientOptions: {'api_endpoint': 'us-central1-aiplatform.googleapis.com', 'client_cert_source': None, 'client_encrypted_cert_source': None, 'quota_project_id': None, 'credentials_file': None, 'scopes': None, 'api_key': None, 'api_audience': None, 'universe_domain': None}, api_endpoint=None, api_transport=None, default_metadata=(), additional_headers=None, client_cert_source=None, credentials=None, client_preview=None, temperature=None, max_output_tokens=None, top_p=None, top_k=None, n=1, streaming=False, model_family=None, safety_settings=None, tuned_model_name=None, instance={'max_batch_size': 250, 'batch_size': 250, 'min_batch_size': 5, 'min_good_batch_size': 5, 'lock': <unlocked _thread.lock object at 0x3480911

In [12]:
# Remove empty chunks 

from langchain.schema import Document

def remove_empty_chunks(chunks_start: list) -> list:
    
    start = len(chunks_start)
    # print(f'start - {start} chunks')
    
    # Remove empty chunks
    chunks_end = [chunk for chunk in chunks_start if chunk.page_content.strip()]

    end = len(chunks_end)
    # print(f'end - {end} chunks')

    return chunks_end   

#####

def test_remove_empty_chunks():
    docs = process_file('docs/medi-cal/acu.pdf')

    # Added a test doc
    doc = Document(
        page_content='',
        metadata=docs[0].metadata
    )
    docs.append(doc)
    print(len(docs))

    # Remove the empty doc (chunk)
    docs = remove_empty_chunks(docs)
    print(len(docs))

test_remove_empty_chunks()

5
4


In [13]:
# Create a text splitter using recursive character text splitter

# https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/recursive_text_splitter/

from langchain.text_splitter import RecursiveCharacterTextSplitter

def chunk_docs_recursive(documents: list, chunk_size=500, chunk_overlap=50) -> list:

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    chunks_start = text_splitter.split_documents(documents)

    chunks_end = remove_empty_chunks(chunks_start=chunks_start)

    return chunks_end

#####

def test_chunk_docs_recursive(): 
    docs = process_file('docs/medi-cal/acu.pdf')
    chunks = chunk_docs_recursive(documents=docs)
    print(len(chunks))
    print(chunks[0].page_content)

test_chunk_docs_recursive()

13
acu 
1 
Part 2 – Acupuncture Services 
Acupuncture Services 
Page updated: August 2020 
This section contains information about acupuncture services and program coverage 
(California Code of Regulations [CCR], Title 22, Section 51308.5).  Senate Bill 833 
(Committee on Budget and Fiscal Review, Chapter 30, Statutes of 2016) restored 
acupuncture as a Medi-Cal benefit for all eligible beneficiaries, effective July 1, 2016.  For


In [14]:
# Create a text splitter using NLTK

# https://python.langchain.com/docs/how_to/split_by_token/

from langchain.text_splitter import NLTKTextSplitter

def chunk_docs_nltk(documents: list, chunk_size=500, chunk_overlap=50) -> list:

    text_splitter = NLTKTextSplitter(
    chunk_size=chunk_size, 
    chunk_overlap=chunk_overlap)

    chunks_start = text_splitter.split_documents(documents)

    chunks_end = remove_empty_chunks(chunks_start=chunks_start)

    return chunks_end

#####

def test_chunk_docs_nltk(): 
    docs = process_file('docs/medi-cal/acu.pdf')
    chunks = chunk_docs_nltk(documents=docs)
    print(len(chunks))
    print(chunks[0].page_content)

test_chunk_docs_nltk()


Created a chunk of size 983, which is longer than the specified 500


12
acu 
1 
Part 2 – Acupuncture Services 
Acupuncture Services 
Page updated: August 2020 
This section contains information about acupuncture services and program coverage 
(California Code of Regulations [CCR], Title 22, Section 51308.5).

Senate Bill 833 
(Committee on Budget and Fiscal Review, Chapter 30, Statutes of 2016) restored 
acupuncture as a Medi-Cal benefit for all eligible beneficiaries, effective July 1, 2016.


In [15]:
# Create a text splitter semantic chunking 

# https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/semantic-chunker/

from langchain_experimental.text_splitter import SemanticChunker

def chunk_docs_semantic(documents: list, ) -> list:

    # TODO - Use embeddings parameter
    text_splitter = SemanticChunker(create_embeddings_openai(), breakpoint_threshold_type="percentile")

    chunks_start = text_splitter.split_documents(documents)

    # Remove empty chunks
    chunks_end = remove_empty_chunks(chunks_start)

    return chunks_end

#####

def test_chunk_docs_semantic():
    docs = process_file('docs/medi-cal/acu.pdf')
    chunks = chunk_docs_semantic(docs)
    print(len(chunks))
    print(chunks[0].page_content)

test_chunk_docs_semantic()

7
acu 
1 
Part 2 – Acupuncture Services 
Acupuncture Services 
Page updated: August 2020 
This section contains information about acupuncture services and program coverage 
(California Code of Regulations [CCR], Title 22, Section 51308.5). Senate Bill 833 
(Committee on Budget and Fiscal Review, Chapter 30, Statutes of 2016) restored 
acupuncture as a Medi-Cal benefit for all eligible beneficiaries, effective July 1, 2016. For 
additional help, refer to the acupuncture services billing example section in this manual. Program Coverage 
Acupuncture services are reimbursable only when: 
• Rendered by a physician, dentist, podiatrist or certified acupuncturist enrolled in the 
Medi-Cal program and who is eligible to provide Medi-Cal services. • Limited to treatment performed to prevent, modify or alleviate the perception of severe, 
persistent chronic pain resulting from a generally recognized medical condition. • Used with or without electric stimulation of the needles. • Used to treat a 

In [16]:
# Create a Qdrant vector store

from langchain_core.embeddings import Embeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

def create_qdrant_vector_store(location: str, collection_name: str, vector_size: int, embeddings: Embeddings, documents: list) -> QdrantVectorStore:

    # Initialize the Qdrant client
    qdrant_client = QdrantClient(location=location)

    # Create a collection in Qdrant
    qdrant_client.create_collection(collection_name=collection_name, vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE))

    # Initialize QdrantVectorStore with the Qdrant client
    qdrant_vector_store = QdrantVectorStore(client=qdrant_client, collection_name=collection_name, embedding=embeddings)
    
    qdrant_vector_store.add_documents(documents)
    
    return qdrant_vector_store

#####

def test_create_qdrant_vector_store():
    embeddings = create_embeddings_openai()
    docs = process_file('docs/medi-cal/acu.pdf')
    print(len(docs))
    chunks = chunk_docs_recursive(docs)
    print(len(chunks))
    vector_store = create_qdrant_vector_store(":memory:", "test", 1536, embeddings, chunks)
    print(vector_store.collection_name)

test_create_qdrant_vector_store()

4
13
test


In [17]:
# Create a Qdrant retriever

# TODO - Add reference 

from langchain_core.retrievers import BaseRetriever
from langchain_qdrant import QdrantVectorStore

def create_retriever_qdrant(vector_store: QdrantVectorStore) -> BaseRetriever:

    retriever = vector_store.as_retriever()

    return retriever

#####

def test_create_retriever_qdrant(text: str = None):
    embeddings = create_embeddings_openai()
    docs = process_file('docs/medi-cal/acu.pdf')
    chunks = chunk_docs_recursive(docs)
    vector_store = create_qdrant_vector_store(":memory:", "test", 1536, embeddings, chunks)
    retriever = create_retriever_qdrant(vector_store)
    if text:
        docs = retriever.invoke(text)
        print(docs[0])

print('\nQDRANT')
test_create_retriever_qdrant('What is my benefit for acupuncture?')


QDRANT
page_content='additional help, refer to the acupuncture services billing example section in this manual. 
Program Coverage 
Acupuncture services are reimbursable only when: 
• Rendered by a physician, dentist, podiatrist or certified acupuncturist enrolled in the 
Medi-Cal program and who is eligible to provide Medi-Cal services. 
• Limited to treatment performed to prevent, modify or alleviate the perception of severe, 
persistent chronic pain resulting from a generally recognized medical condition.' metadata={'source': 'docs/medi-cal/acu.pdf', 'file_path': 'docs/medi-cal/acu.pdf', 'page': 0, 'total_pages': 4, 'format': 'PDF 1.7', 'title': 'Acupuncture Services (acu)', 'author': 'Department of Healthcare Services (DHCS)', 'subject': '', 'keywords': 'acu_a01o11', 'creator': 'Microsoft® Word for Office 365', 'producer': 'Microsoft® Word for Office 365', 'creationDate': "D:20200731144705-07'00'", 'modDate': "D:20200731155009-07'00'", 'trapped': '', '_id': '088c6fc728cc46839fd4a15

In [18]:
# Create a Vertex AI retriever

# https://python.langchain.com/docs/integrations/retrievers/google_vertex_ai_search/

from langchain_google_community import VertexAISearchRetriever

def create_retriever_vertexai() -> VertexAISearchRetriever:

    retriever = VertexAISearchRetriever(project_id=os.environ['PROJECT_ID'], location_id=os.environ['LOCATION_ID'], data_store_id=os.environ['DATA_STORE_ID'], max_documents=3)

    return retriever

#####

def test_create_retriever_vertexai(text: str = None):
    retriever = create_retriever_vertexai()
    if text:
        docs = retriever.invoke(text)
        print(docs[0])

print('\nVERTEX AI')
test_create_retriever_vertexai('What is my benefit for acupuncture?')


VERTEX AI
page_content='# Acupuncture Services

# acu

1

## Page updated: August 2020

This section contains information about acupuncture services and program coverage (California Code of Regulations [CCR], Title 22, Section 51308.5). Senate Bill 833 (Committee on Budget and Fiscal Review, Chapter 30, Statutes of 2016) restored acupuncture as a Medi-Cal benefit for all eligible beneficiaries, effective July 1, 2016. For additional help, refer to the acupuncture services billing example section in this manual.

## Program Coverage

Acupuncture services are reimbursable only when: • Rendered by a physician, dentist, podiatrist or certified acupuncturist enrolled in the Medi-Cal program and who is eligible to provide Medi-Cal services. • Limited to treatment performed to prevent, modify or alleviate the perception of severe, persistent chronic pain resulting from a generally recognized medical condition. • Used with or without electric stimulation of the needles. • Used to treat a cond

In [19]:
# Create a prompt template

# https://python.langchain.com/v0.1/docs/modules/model_io/prompts/quick_start/#chatprompttemplate
# https://python.langchain.com/v0.2/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html

from langchain.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate

def create_chat_prompt_template(prompt: str = None) -> ChatPromptTemplate:
    
    template = '''
    You are a helpful conversational agent for the State of California.
    Your expertise is fully understanding the Medi-Cal provider manuals. 
    You need to answer questions posed by a member, who is trying to get answers about services provided by Medi-Cal.  
    Your goal is to provide a helpful and detailed response, in at least 2-3 sentences. 

    You will be analyzing the health plan documents to derive a good answer, based on the following information:
    1. The question asked.
    2. The provided context, which comes from health plan document. You will need to answer the question based on the provided context and the conversation history.

    Now it's your turn!

    {question}

    {context}
    '''
    
    prompt = PromptTemplate.from_template(template)

    return prompt

#####

def test_create_chat_prompt_template():
    prompt = create_chat_prompt_template()
    print(prompt)

test_create_chat_prompt_template()

input_variables=['context', 'question'] template="\n    You are a helpful conversational agent for the State of California.\n    Your expertise is fully understanding the Medi-Cal provider manuals. \n    You need to answer questions posed by a member, who is trying to get answers about services provided by Medi-Cal.  \n    Your goal is to provide a helpful and detailed response, in at least 2-3 sentences. \n\n    You will be analyzing the health plan documents to derive a good answer, based on the following information:\n    1. The question asked.\n    2. The provided context, which comes from health plan document. You will need to answer the question based on the provided context and the conversation history.\n\n    Now it's your turn!\n\n    {question}\n\n    {context}\n    "


In [20]:
# Create a Langchain chain..

# https://python.langchain.com/docs/integrations/llms/google_ai/
# https://python.langchain.com/docs/integrations/chat/google_generative_ai/
# https://ai.google.dev/gemini-api/docs/safety-settings 

from google.generativeai.types import HarmCategory, HarmBlockThreshold
from langchain_core.output_parsers import StrOutputParser
from langchain_core.retrievers import BaseRetriever
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI
from operator import itemgetter

def create_chain (model_name: str, prompt: ChatPromptTemplate, retriever: BaseRetriever):

    if "gemini" in model_name.lower():
        llm = ChatGoogleGenerativeAI(
            model=model_name,
            temperature=0,
            safety_settings={
                    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                },
            )
    else:
        print("Unsuported model name")
        
    chain = (
        {"context": itemgetter("question") | retriever, "question": itemgetter("question")} 
        | RunnablePassthrough.assign(context=itemgetter("context")) 
        | {"response": prompt | llm, "context": itemgetter("context")}
        )

    return chain

#####

def test_create_chain_qdrant():
    embeddings = create_embeddings_openai()
    docs = process_file('docs/medi-cal/acu.pdf')
    chunks = chunk_docs_recursive(docs)
    vector_store = create_qdrant_vector_store(":memory:", "test", 1536, embeddings, chunks)
    retriever = create_retriever_qdrant(vector_store)
    chat_prompt_template = create_chat_prompt_template()
    chain = create_chain('gemini-1.5-flash', chat_prompt_template, retriever)
    result = chain.invoke({'question' : 'What is my benefit for acupuncture?'})
    print(result)

print('\nQDRANT')
test_create_chain_qdrant()

###

def test_create_chain_vertexai():
    retreiver = create_retriever_vertexai()
    chat_prompt_template = create_chat_prompt_template()
    chain = create_chain('gemini-1.5-flash', chat_prompt_template, retreiver)
    result = chain.invoke({'question' : 'What is my benefit for acupuncture?'})
    print(result)

print('\nVERTEX AI')
test_create_chain_vertexai()


QDRANT
{'response': AIMessage(content='Medi-Cal covers acupuncture services for the treatment of severe, persistent chronic pain stemming from a generally recognized medical condition.  However,  the services must be provided by a physician, dentist, podiatrist, or certified acupuncturist enrolled in Medi-Cal.  There is a limit of two acupuncture services per calendar month, or a combination of two services per month from a list of specified therapies including acupuncture, audiology, chiropractic, occupational therapy, podiatry, and speech therapy.\n', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-980eb57d-85b1-4089-be70-abe1ee898821-0', usage_metadata={'input_tokens': 1515, 'output_tokens': 101, 'total_tokens': 1616}), 'context': [Document(metadata={'source': 'docs/medi-cal/acu.pdf', 'file_path': 'docs/medi-cal/acu.pdf', 'page': 0, 'total_pages': 4, 'format': 'PDF 1.7', 'title': 'Acupuncture S

In [21]:
# Generate answers from a chain usin a list of questions

def generate_answers_contexts(chain, questions: list):
    
    answers = []
    contexts = []

    # Loop over the list of questions and call the chain to get the answer and context
    for question in questions:
        print(question)

        # Call the chain to get answers and contexts
        response = chain.invoke({"question" : question})
        print(response)
        
        # Capture the answer and context 
        answers.append(response["response"].content)
        contexts.append([context.page_content for context in response["context"]])

    return answers, contexts

#####

# TODO - Add test

In [ ]:
# Run a Ragas evaluation 

from datasets import Dataset
from pandas import DataFrame
from ragas import evaluate
from ragas.metrics import (faithfulness, answer_relevancy, answer_correctness, context_recall, context_precision)

def run_ragas_evaluation(chain, questions: list, groundtruths: list, eval_metrics: list = [answer_correctness, answer_relevancy, context_recall, context_precision, faithfulness]):

  answers = []
  contexts = []
  answers, contexts = generate_answers_contexts(chain, questions)

  # Create the input dataset 
  input_dataset = Dataset.from_dict({
  "question" : questions,         # From the dataframe
  "answer" : answers,             # From the chain
  "contexts" : contexts,          # From the chain
  "ground_truth" : groundtruths   # From the dataframe
  })

  # Run the Ragas evaluation using the input dataset and eval metrics
  ragas_results = evaluate(input_dataset, eval_metrics)
  ragas_results_df = ragas_results.to_pandas()
  
  return ragas_results, ragas_results_df
  
#####

def test_run_ragas_evaluation():
  print("test")    

test_run_ragas_evaluation()

test


In [23]:
# Query Vertex AI Search datastore using HTTP Post

import requests
import google.auth
from google.auth.transport.requests import Request

credentials, project_id = google.auth.default()
credentials.refresh(Request())
access_token = credentials.token
print(access_token)

def query_chunks(query, n=5):
    
  PROJECT_ID = os.environ['PROJECT_ID']
  LOCATION_ID = os.environ['LOCATION_ID']
  DATA_STORE_ID = os.environ['DATA_STORE_ID']

  if LOCATION_ID == 'us':
    api_endpoint = 'us-discoveryengine.googleapis.com'
  else:
    api_endpoint = 'discoveryengine.googleapis.com'

  url = f"https://{api_endpoint}/v1alpha/projects/{PROJECT_ID}/locations/{LOCATION_ID}/collections/default_collection/dataStores/{DATA_STORE_ID}/servingConfigs/default_search:search"
  print(url)
  
  headers = {
      "Authorization": f"Bearer {access_token}",
      "Content-Type": "application/json",
  }
  
  post_data = {
      "servingConfig": f"projects/{PROJECT_ID}/locations/{LOCATION_ID}/collections/default_collection/dataStores/{DATA_STORE_ID}/servingConfigs/default_search",
      "pageSize": n,
      "query": query,
      "contentSearchSpec": {"searchResultMode": "CHUNKS"},
  }
  
  response = requests.post(url, headers=headers, json=post_data)

  if response.status_code != 200:
    print(
        f"Error retrieving search results: {response.status_code} -"
        f" {response.text}"
    )

  return response.json()

#####

test = query_chunks('What is my benefit for acupuncture?')
print(test)

ya29.a0AeDClZAXlcwZKa5doxJttn0fbQCi4fMo4K0_aYTmRZzJ-54wW-ABJiCe8oDx4CrdRqPn19zsQlCrDuHwzfAEJbh5xXi5nezAsVCIA5j57tFztU1ilKH0AYIhaiGc-5hWV0l88vPH8Nq4IlHoPD5E8gzt3x8CYU3B6FmWQCJIDJ4aCgYKAaMSARESFQHGX2MiW2YSfQQ5luR3Q1LJbUmPlA0178
https://us-discoveryengine.googleapis.com/v1alpha/projects/aie4-demo/locations/us/collections/default_collection/dataStores/aie4-demo-medi-cal-ds_1729613223026/servingConfigs/default_search:search
{'results': [{'chunk': {'name': 'projects/395640738565/locations/us/collections/default_collection/dataStores/aie4-demo-medi-cal-ds_1729613223026/branches/0/documents/7fcb9cd8371b20fd91ce9f0daf8b2cb7/chunks/c1', 'id': 'c1', 'content': "# Acupuncture Services\n\n# acu\n\n1\n\n## Page updated: August 2020\n\nThis section contains information about acupuncture services and program coverage (California Code of Regulations [CCR], Title 22, Section 51308.5). Senate Bill 833 (Committee on Budget and Fiscal Review, Chapter 30, Statutes of 2016) restored acupuncture as a Medi-Cal

# Create RAG chain using Google Vertex AI

In [24]:
# Build RAG chain using Vertex AI Agent Builder datastore

retreiver = create_retriever_vertexai()
chat_prompt_template = create_chat_prompt_template()
chain = create_chain('gemini-1.5-flash', chat_prompt_template, retreiver)

In [25]:
# Test the chain 

questions = ['What is my benefit for acupuncture?',
'Who should I call if I have an emergency?',
'Provide an overview of the newborn hearing screening program?',]

for question in questions:
    print(question)
    result = chain.invoke({"question" : question})
    print(result)
    print(result["response"].content)

What is my benefit for acupuncture?
{'response': AIMessage(content='Your Medi-Cal benefit for acupuncture is limited to treatment for severe, persistent chronic pain stemming from a generally recognized medical condition.  This treatment must be performed by a Medi-Cal enrolled physician, dentist, podiatrist, or certified acupuncturist.  The service is subject to a two-visit-per-month limit and may be used with or without electrical stimulation, and in conjunction with other covered treatment modalities.  Reimbursement rates are capped at $20.00 for each 15-minute session, whether with or without electrical stimulation.\n', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-990a70f4-b2e9-4629-b658-73401bcda51a-0', usage_metadata={'input_tokens': 2169, 'output_tokens': 116, 'total_tokens': 2285}), 'context': [Document(metadata={'id': '7fcb9cd8371b20fd91ce9f0daf8b2cb7', 'source': 'gs://aie4-demo-medi-ca

# Create synthetic testset

In [27]:
# https://docs.ragas.io/en/latest/references/generate/

from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from ragas.testset.evolutions import simple, reasoning, multi_context
from ragas.testset.generator import TestsetGenerator
from ragas.testset.generator import RunConfig

# Load the docs
docs = []
docs_pdf = process_directory('docs/medi-cal/', '**/*.pdf', PyMuPDFLoader, True)
docs.extend(docs_pdf)
docs_excel = process_directory('docs/medi-cal/', '**/*.xlsx', UnstructuredExcelLoader, True)
docs.extend(docs_excel)
print(len(docs))
    
# # Chunk the docs 
# # chunks = chunk_docs_nltk(docs, 1500, 150)
# chunks = chunk_docs_semantic(docs)

# Set up the parameters for generating the testset
generator_llm = ChatOpenAI(model="gpt-4o-mini")
critic_llm = ChatOpenAI(model="gpt-4o-mini")
embeddings = OpenAIEmbeddings()
run_config=RunConfig(
        # timeout=60,
        # max_retries=10,
        # max_wait = 180, # default: 60
        max_workers=4, # default: 16
        )
generator = TestsetGenerator.from_langchain(generator_llm=generator_llm, critic_llm=critic_llm, embeddings=embeddings,run_config=run_config)
distributions = {simple: 0.5, multi_context: 0.4, reasoning: 0.1}

# Generate the testset and save to disk 
testset = generator.generate_with_langchain_docs(documents=docs, test_size=25, distributions=distributions)
testset_df = testset.to_pandas()

print("Writing synthetic_testset.csv")
testset_df.to_csv("testsets/synthetic_testset.csv")
testset_df

  0%|          | 0/1 [00:00<?, ?it/s]/opt/anaconda3/envs/aie4/lib/python3.11/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/opt/anaconda3/envs/aie4/lib/python3.11/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
100%|██████████| 1/1 [00:00<00:00, 11.72it/s]


2477


Filename and doc_id are the same for all nodes.                     
Generating:   0%|          | 0/25 [04:32<?, ?it/s]


KeyboardInterrupt: 

# Evaluate synthetic dataset using Ragas

In [ ]:
# Evaluate the synthetic testset using Ragas

import pandas as pd

# Run the Ragas evaluation and show the results
# Get the questions and groundtruths from the dataframe
testset_df = pd.read_csv("testsets/synthetic_testset.csv")

questions = testset_df["question"].values.tolist()
questions = [str(question) for question in questions]

groundtruths = testset_df["ground_truth"].values.tolist()
groundtruths = [str(ground_truth) for ground_truth in groundtruths]  

# Specify the eval metrics
eval_metrics = [answer_correctness, answer_relevancy, context_precision, context_recall, faithfulness]

# Run the Ragas evaluation and show the results
ragas_results, ragas_results_df = run_ragas_evaluation(chain, questions, groundtruths, eval_metrics)

# Write the results to disk
print("Writing synthetic_testset_ragas_results.csv")
ragas_results_df.to_csv("ragas/synthetic_testset_ragas_results.csv")

# Show the resutls
ragas_results

# Create sample ReAct agent using Gemini

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, load_tools
from langchain_google_genai import ChatGoogleGenerativeAI
from google.generativeai.types import HarmCategory, HarmBlockThreshold

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", 
                             temperature=0, 
                             safety_settings={
                                 HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                                 HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                                 HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                                 HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH
                                 }
                                 )

tools = load_tools(["arxiv"])

prompt = hub.pull("hwchase17/react")

agent = create_react_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "What's the paper 1605.08386 about?"})

# Use Unstructured to parse docs and load to Faiss

In [ ]:
# https://colab.research.google.com/drive/1BJYYyrPVe0_9EGyXqeNyzmVZDrCRZwsg?usp=sharing#scrollTo=CwzrH-9_K6-z

%pip install -qU "unstructured-ingest[pdf]" unstructured faiss-cpu

In [24]:
from unstructured_ingest.v2.pipeline.pipeline import Pipeline
from unstructured_ingest.v2.interfaces import ProcessorConfig
from unstructured_ingest.v2.processes.connectors.local import (
    LocalIndexerConfig,
    LocalDownloaderConfig,
    LocalConnectionConfig,
    LocalUploaderConfig
)
from unstructured_ingest.v2.processes.partitioner import PartitionerConfig
from unstructured_ingest.v2.processes.chunker import ChunkerConfig

In [25]:
import os 

directory_with_pdfs="app_2/content/data"
directory_with_results="app_2/content/output"

In [ ]:
Pipeline.from_configs(
    context=ProcessorConfig(),
    indexer_config=LocalIndexerConfig(input_path=directory_with_pdfs),
    downloader_config=LocalDownloaderConfig(),
    source_connection_config=LocalConnectionConfig(),
    partitioner_config=PartitionerConfig(
        partition_by_api=True,
        api_key=os.getenv("UNSTRUCTURED_API_KEY"),
        partition_endpoint=os.getenv("UNSTRUCTURED_API_URL"),
        strategy="hi_res",
        additional_partition_args={
            "split_pdf_page": True,
            "split_pdf_concurrency_level": 15,
            },
        ),
    uploader_config=LocalUploaderConfig(output_dir=directory_with_results)
).run()

In [ ]:
from unstructured.staging.base import elements_from_json

elements = []
for filename in os.listdir(directory_with_results):
    if filename.endswith('.json'):
        file_path = os.path.join(directory_with_results, filename)
        try:
            elements.extend(elements_from_json(filename=file_path))
        except IOError:
            print(f"Error: Could not read file {filename}.")
print(len(elements))

In [ ]:
from langchain.schema import Document

docs = []

for element in elements:
    metadata = element.metadata.to_dict()
    docs.append(Document(page_content=element.text, metadata=metadata))

print(len(docs))

In [ ]:
from langchain.vectorstores import FAISS

vector_store = FAISS.from_documents(documents=docs, embedding=create_embeddings_vertexai())
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [36]:
vector_store.save_local("app_2/faiss_index_unstructured")

In [ ]:
new_vector_store = FAISS.load_local("app_2/faiss_index_unstructured", create_embeddings_vertexai(), allow_dangerous_deserialization=True)
new_retriever = new_vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [41]:
new_chain = create_chain('gemini-1.5-flash', chat_prompt_template, new_retriever)

In [ ]:
questions = ['What is my benefit for acupuncture?',
'Who should I call if I have an emergency?',
'What is my vision benefit?',]

for question in questions:
    print(question)
    result = chain.invoke({"question" : question})
    print(result)
    print(result["response"].content)

# Using PyMuPDFLoader to parse docs and load to Faiss

In [ ]:
docs = []

docs_pdf = process_directory('docs/medi-cal/', '**/*.pdf', PyMuPDFLoader, True)
docs_excel = process_directory('docs/medi-cal/', '**/*.xlsx', UnstructuredExcelLoader, True)

docs.extend(docs_pdf)
docs.extend(docs_excel)

print(len(docs))



In [60]:
from langchain.vectorstores import FAISS

vector_store = FAISS.from_documents(documents=docs, embedding=create_embeddings_vertexai())
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [56]:
vector_store.save_local("app_2/faiss_index_pymupdfloader")

In [61]:
new_vector_store = FAISS.load_local("app_2/faiss_index_pymupdfloader", create_embeddings_vertexai(), allow_dangerous_deserialization=True)
new_retriever = new_vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [69]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-english-v3.0")
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=new_retriever)

In [70]:
template = '''
You are a helpful conversational agent for the State of California.
Your expertise is fully understanding the Medi-Cal provider manuals. 
You need to answer questions posed by a member, who is trying to get answers about services provided by Medi-Cal.  
Your goal is to provide a helpful and detailed response, in at least 2-3 sentences. 

You will be analyzing the health plan documents to derive a good answer, based on the following information:
1. The question asked.
2. The provided context, which comes from health plan document. You will need to answer the question based on the provided context and the conversation history.

Now it's your turn!

{question}

{context}
'''

prompt = PromptTemplate.from_template(template)

llm = ChatGoogleGenerativeAI(
    model='gemini-1.5-flash',
    temperature=0,
    safety_settings={
            HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
            HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
            HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
            HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        }
    )
        
# new_chain = (
#         {"context": itemgetter("question") | new_retriever, "question": itemgetter("question")} 
#         | RunnablePassthrough.assign(context=itemgetter("context")) 
#         | {"response": prompt | llm, "context": itemgetter("context")}
#         )

new_chain = (
        {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")} 
        | RunnablePassthrough.assign(context=itemgetter("context")) 
        | {"response": prompt | llm, "context": itemgetter("context")}
        )


In [ ]:
questions = ['What is my benefit for acupuncture?',
'Who should I call if I have an emergency?',
'What is my vision benefit?',]

for question in questions:
    print(question)
    result = new_chain.invoke({"question" : question})
    print(result)
    print(result["response"].content)